In [ ]:
!pip install datasets

In [ ]:
!pip install openai>=1.0

In [ ]:
from huggingface_hub import login
login(
token="xxxxxx", # ADD YOUR TOKEN HERE
add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset
dataset = load_dataset("CAII-NCSA/ECE408_FA23_first_interactions")
print(dataset['train'][0].keys())

Generating train split:   0%|          | 0/1072 [00:00<?, ? examples/s]

dict_keys(['first_query', 'first_contexts', 'contexts', 'full_prompt', 'first_response', 'references', 'response_without_references', '__index_level_0__'])


In [ ]:
print("________________this is first query_________________")
print(dataset['train'][0]["first_query"])
print("\n")
print("________________this is first contexts_________________")
print(dataset['train'][0]["first_contexts"])
print("\n")
# print("________________this is full prompt_________________")
# print(dataset['train'][0]['full_prompt'])
# print("\n")
print("________________this is first response_________________")
print(dataset['train'][0]['first_response'])
print("\n")
print("________________this is references_________________")
print(dataset['train'][0]['references'])
print("\n")
print("________________this is response_without_references_________________")
print(dataset['train'][0]['response_without_references'])

________________this is first query_________________
Explain how tiling helps with global memory bandwidth.


________________this is first contexts_________________
[{'base_url': '', 'course_name ': 'ECE408SP24', 'pagenumber': 57, 'readable_filename': 'CUDA_C_Best_Practices_Guide.pdf', 's3_path': 'courses/ECE408SP24/f1ffcfe0-43a3-42bd-a60b-94fdbab7914f-CUDA_C_Best_Practices_Guide.pdf', 'text': 'CUDA C++ Best Practices Guide, Release 12.3\nFig. 10: The performance of the sliding-window benchmark with tuned hit-ratio\n13.2.3. Shared Memory\nBecause it is on-chip, shared memory has much higher bandwidth and lower latency than local and\nglobal memory - provided there are no bank conflicts between the threads, as detailed in the following\nsection.\n13.2.3.1 Shared Memory and Memory Banks\nTo achieve high memory bandwidth for concurrent accesses, shared memory is divided into equally\nsized memory modules (banks) that can be accessed simultaneously. Therefore, any memory load or\nstore of

In [ ]:
print("________________this is full prompt_________________")
print(dataset['train'][0]['full_prompt'])
print("\n")

________________this is full prompt_________________

Please analyze and respond to the following question using the excerpts from the provided documents. These documents can be pdf files or web pages.
    Integrate relevant information from these documents, ensuring each reference is linked to the document's number.
    Your response should be semi-formal. 
    When quoting directly, cite with footnotes linked to the document number and page number, if provided. 
    Summarize or paraphrase other relevant information with inline citations, again referencing the document number and page number, if provided.
    If the answer is not in the provided documents, state so. 
    Conclude your response with a LIST of the document titles as clickable placeholders, each linked to its respective document number and page number, if provided.
    Always share page numbers if they were shared with you.
    ALWAYS follow the examples below:
    Insert an inline citation like this in your response: 


In [ ]:
print(len(dataset["train"]))

1072


In [ ]:
import pandas as pd
first_query = dataset['train']["first_query"]
first_response = dataset['train']["first_response"]
full_prompt = dataset['train']['full_prompt']


# Create a dictionary of lists
data = {'first_query': first_query,
     'first_response': first_response,
     "full_prompt": full_prompt}


# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

print(df.head())

                                         first_query  \
0  Explain how tiling helps with global memory ba...   
1      Show an example of [CUDA API] function usage.   
2  what is my chance to get into ece master at ui...   
3  Explain how tiling helps with global memory ba...   
4                    When is the first midterm exam?   

                                      first_response  \
0  Tiling is a technique used in CUDA programming...   
1  Sure, here are a few examples of CUDA API func...   
2  I'm sorry, but the documents provided do not c...   
3  Tiling is a technique that helps optimize the ...   
4  The first midterm exam is scheduled after appr...   

                                         full_prompt  
0  \nPlease analyze and respond to the following ...  
1  \nPlease analyze and respond to the following ...  
2  \nPlease analyze and respond to the following ...  
3  \nPlease analyze and respond to the following ...  
4  \nPlease analyze and respond to the following ..

In [ ]:
print(len(df["first_query"].unique()))
print(len(df["first_query"]))

975
1072


In [ ]:
test = df[df["first_query"] == "explain tiles matrix multiplication"]
print(test)

                             first_query  \
780  explain tiles matrix multiplication   

                                        first_response  \
780  Tiled matrix multiplication is an optimization...   

                                           full_prompt  \
780  \nPlease analyze and respond to the following ...   

                                              critique  
780  af3570b60c2c121304683a6ea099da58b26a109f8beaec...  


In [ ]:
from huggingface_hub import InferenceClient
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=model_id,
    timeout=120,
)


import pandas as pd

for i, row in df.iterrows():
    first_query = row["first_query"]
    first_response = row["first_response"]

    system_prompt = f"""You are a brilliant AI assistant.
    You will receive an input question and the model-generated answer.
    You need to generate the specific and actionable critiques for the model-generated answer,
    which contain the critical comments and helpful suggestions."""

    user_prompt = f"""input question: {first_query}
                    model-generated answer: {first_response}.
                    critique:"""

    full_prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>{user_prompt} [/INST]\n\n"
    output = llm_client.text_generation(prompt=full_prompt, max_new_tokens=1000)

    df.at[i, 'critique'] = output
    print(f"generated the {i} sample")
    df.to_json('critic_output.json', orient='records', lines=True)








generated the 0 sample
generated the 1 sample
generated the 2 sample
generated the 3 sample
generated the 4 sample
generated the 5 sample
generated the 6 sample
generated the 7 sample
generated the 8 sample
generated the 9 sample
generated the 10 sample
generated the 11 sample
generated the 12 sample
generated the 13 sample
generated the 14 sample
generated the 15 sample
generated the 16 sample
generated the 17 sample
generated the 18 sample
generated the 19 sample
generated the 20 sample
generated the 21 sample
generated the 22 sample
generated the 23 sample
generated the 24 sample
generated the 25 sample
generated the 26 sample
generated the 27 sample
generated the 28 sample
generated the 29 sample
generated the 30 sample
generated the 31 sample
generated the 32 sample
generated the 33 sample
generated the 34 sample
generated the 35 sample
generated the 36 sample
generated the 37 sample
generated the 38 sample
generated the 39 sample
generated the 40 sample
generated the 41 sample
ge

In [ ]:
import pandas as pd

# Read the JSON file into a DataFrame
df = pd.read_json('critic_output.json', orient='records', lines=True)

print(df.loc[277]['critique'])

None


In [ ]:
print(df.shape)

(1072, 4)


The input maybe too long so that we need to set a upper bound for that. The following code shows how to do that.

In [ ]:
from huggingface_hub import InferenceClient
from transformers import AutoTokenizer
import json
import pandas as pd

# Initialize the tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# tokenizer.model_max_length = 32768
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")

# Define a maximum token limit for the input
max_input_tokens = 32768-1000-10  # Reserve some tokens for the output and the extra tokens

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=model_id,
    timeout=120,
)

for i, row in df.iterrows():
  if i >= 781:
    first_query = row["first_query"]
    first_response = row["first_response"]

    system_prompt = f"""You are a brilliant AI assistant.
    You will receive an input question and the model-generated answer.
    You need to generate the specific and actionable critiques for the model-generated answer,
    which contain the critical comments and helpful suggestions."""

    user_prompt = f"""input question: {first_query}
                    model-generated answer: {first_response}.
                    critique:"""

    full_prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>{user_prompt} [/INST]\n\n"

     # Tokenize the input text
    input_tokens = tokenizer.encode(full_prompt)

    # If the input text is too long, truncate it
    if len(input_tokens) > max_input_tokens:
        input_tokens = input_tokens[:max_input_tokens]

    # Convert the tokens back to text
    truncated_prompt = tokenizer.decode(input_tokens)

    # Generate the output
    output = llm_client.text_generation(prompt=truncated_prompt, max_new_tokens=1000)


    df.at[i, 'critique'] = output
    print(f"generated the {i} sample")
     # Convert the row to a dictionary
    row_dict = df.loc[i].to_dict()

    # Append the row to the JSON file
    with open('/content/drive/MyDrive/ECE_data/critic_output_part4.json', 'a') as f:
        f.write(json.dumps(row_dict) + '\n')

generated the 781 sample
generated the 782 sample
generated the 783 sample
generated the 784 sample
generated the 785 sample
generated the 786 sample
generated the 787 sample
generated the 788 sample
generated the 789 sample
generated the 790 sample
generated the 791 sample
generated the 792 sample
generated the 793 sample
generated the 794 sample
generated the 795 sample
generated the 796 sample
generated the 797 sample
generated the 798 sample
generated the 799 sample
generated the 800 sample
generated the 801 sample
generated the 802 sample
generated the 803 sample
generated the 804 sample
generated the 805 sample
generated the 806 sample
generated the 807 sample
generated the 808 sample
generated the 809 sample
generated the 810 sample
generated the 811 sample
generated the 812 sample
generated the 813 sample
generated the 814 sample
generated the 815 sample
generated the 816 sample
generated the 817 sample
generated the 818 sample
generated the 819 sample
generated the 820 sample
